In [ ]:
import nibabel as nib
import numpy as np
import os

path = "CF6028"
nii_path = "/home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/" + path + ".nii.gz"

img = nib.load(nii_path)
data = img.get_fdata()
data = data.astype(np.int16)

# print all albel
unique_labels = np.unique(data)
print("Unique labels in the segmentation:", unique_labels)

Unique labels in the segmentation: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]


## 1.Turn all the predict results into binary labels

In [13]:
import os
from glob import glob
import nibabel as nib
import numpy as np

TEST_DIR = "/home/cuixing/MDP/Data_Predict/Test_Data"
PRED_DIR = "/home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict"
DST_DIR = PRED_DIR

EXCLUDE_LABELS = (25, 26) # exclude the sturnum and costal cartilage

case_ids = sorted({os.path.basename(f).split('_')[0] for f in glob(os.path.join(TEST_DIR, "*.nii.gz"))})
os.makedirs(DST_DIR, exist_ok=True)

def find_pred(case):
    p = os.path.join(PRED_DIR, f"{case}.nii.gz")
    if os.path.exists(p):
        return p
    candidates = [f for f in glob(os.path.join(PRED_DIR, f"{case}_*.nii.gz")) if not f.endswith("_binary.nii.gz")]
    return sorted(candidates)[0] if candidates else None

ok = miss = 0
for case in case_ids:
    src = find_pred(case)
    if not src:
        print(f"[WARN] {case}: pred not found")
        miss += 1
        continue
    img = nib.load(src)
    data = img.get_fdata()
    mask = (data > 0) & (~np.isin(data, EXCLUDE_LABELS))
    bin_data = mask.astype(np.uint8)
    out_path = os.path.join(DST_DIR, f"{case}_binary.nii.gz")
    out_img = nib.Nifti1Image(bin_data, img.affine, img.header)
    out_img.set_data_dtype(np.uint8)
    nib.save(out_img, out_path)
    print(f"[OK] {case}: {src} -> {out_path}")
    ok += 1

print(f"done saved={ok} missing={miss}")


[OK] CF5008: /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5008.nii.gz -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5008_binary.nii.gz
[OK] CF5009: /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5009.nii.gz -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5009_binary.nii.gz
[OK] CF5015: /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5015.nii.gz -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5015_binary.nii.gz
[OK] CF5017: /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5017.nii.gz -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5017_binary.nii.gz
[OK] CF6008: /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF6008.nii.gz -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF6008_binary.nii.gz
[OK] CF6011: /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF6011.nii.gz -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF6011_

## 2.Test some cases and cauculate there dice score as binary label

In [7]:
path = "CF6028"
pred_path = os.path.join("/home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/", path + "_binary.nii.gz")
gt_path = os.path.join("/home/cuixing/MDP/Ground_Truth", path, "merged_gt.nii.gz")
pred = nib.load(pred_path).get_fdata()
gt = nib.load(gt_path).get_fdata()

pred_count = np.sum(pred > 0)
pred_zero_count = np.sum(pred == 0)
gt_count = np.sum(gt > 0)
gt_zero_count = np.sum(gt == 0)

print("Foreground voxel count in pred:", pred_count)
print("Background voxel count in pred:", pred_zero_count)
print("Foreground voxel count in gt:", gt_count)
print("Background voxel count in gt:", gt_zero_count)


Foreground voxel count in pred: 1014482
Background voxel count in pred: 59791732
Foreground voxel count in gt: 956944
Background voxel count in gt: 59849270


In [8]:
# cauculate for the total Dice score.
def dice_score(y_true, y_pred):
    intersection = np.sum((y_true > 0) & (y_pred > 0))
    sum_volumes = np.sum(y_true > 0) + np.sum(y_pred > 0)
    return (2.0 * intersection) / (sum_volumes + 1e-6)

dice = dice_score(gt, pred)
print(f"Dice Score: {dice}")

Dice Score: 0.9009103055347242


In [7]:
from scipy.ndimage import label
labeled_array, num_features = label(data_new)

print("Detected ribs (connected components):", num_features)

Detected ribs (connected components): 32


In [8]:
sizes = np.bincount(labeled_array.ravel())
for i in range(1, len(sizes)):
    print(f"Label {i}: Size = {sizes[i]} voxels")

Label 1: Size = 40915 voxels
Label 2: Size = 44723 voxels
Label 3: Size = 48585 voxels
Label 4: Size = 24182 voxels
Label 5: Size = 46037 voxels
Label 6: Size = 48959 voxels
Label 7: Size = 40115 voxels
Label 8: Size = 31574 voxels
Label 9: Size = 14110 voxels
Label 10: Size = 25041 voxels
Label 11: Size = 189 voxels
Label 12: Size = 20694 voxels
Label 13: Size = 4413 voxels
Label 14: Size = 2 voxels
Label 15: Size = 51968 voxels
Label 16: Size = 45877 voxels
Label 17: Size = 43472 voxels
Label 18: Size = 2 voxels
Label 19: Size = 39533 voxels
Label 20: Size = 43925 voxels
Label 21: Size = 29973 voxels
Label 22: Size = 17 voxels
Label 23: Size = 4 voxels
Label 24: Size = 16876 voxels
Label 25: Size = 38394 voxels
Label 26: Size = 5459 voxels
Label 27: Size = 29088 voxels
Label 28: Size = 21469 voxels
Label 29: Size = 24441 voxels
Label 30: Size = 278 voxels
Label 31: Size = 16 voxels
Label 32: Size = 5 voxels


# Seperate Label into each ribs.

In [ ]:
import os
import numpy as np
import nibabel as nib
from glob import glob
from scipy.ndimage import label, center_of_mass

TEST_DIR = "/home/cuixing/MDP/Data_Predict/Test_Data"
BIN_DIR  = "/home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict"

def process_case(case):
    src = os.path.join(BIN_DIR, f"{case}_binary.nii.gz")
    if not os.path.exists(src):
        print(f"[MISS] {case}: {src}"); return False

    img = nib.load(src)
    data = (img.get_fdata() > 0).astype(np.uint8)

    labeled_array, _ = label(data)
    sizes = np.bincount(labeled_array.ravel())
    if sizes.size <= 1:
        print(f"[EMPTY] {case}")
        return False

    comp = sizes[1:]
    k = min(24, comp.size)
    topk = np.argsort(comp)[-k:][::-1]

    filt = np.zeros_like(labeled_array, dtype=np.int16)
    for new_lbl, orig_idx in enumerate(topk, start=1):
        filt[labeled_array == (orig_idx + 1)] = new_lbl

    idxs = np.arange(1, k + 1)
    centers = np.array(center_of_mass(filt, labels=filt, index=idxs))  # (x,y,z)

    x_med = np.median(centers[:, 0])
    left_ids  = np.where(centers[:, 0] <  x_med)[0]
    right_ids = np.where(centers[:, 0] >= x_med)[0]

    left_sorted  = left_ids[np.argsort(centers[left_ids, 2])[::-1]]   # Z from top to bottom
    right_sorted = right_ids[np.argsort(centers[right_ids, 2])[::-1]]

    final = np.zeros_like(filt, dtype=np.int16)
    # right 1..12, left 13..24
    for i, idx in enumerate(right_sorted[:12], start=1):
        final[filt == (idx + 1)] = i            # right -> 1..12
    for i, idx in enumerate(left_sorted[:12], start=1):
        final[filt == (idx + 1)] = i + 12       # left -> 13..24

    save_dir = os.path.join(BIN_DIR, case)
    os.makedirs(save_dir, exist_ok=True)

    nib.save(nib.Nifti1Image(final, img.affine, img.header), os.path.join(save_dir, "labeled.nii.gz"))

    for i in range(1, 13):
        m = (final == i).astype(np.int16)
        if m.any():
            nib.save(nib.Nifti1Image(m, img.affine, img.header), os.path.join(save_dir, f"rib_left_{i}.nii.gz"))
    for i in range(1, 13):
        m = (final == i + 12).astype(np.int16)
        if m.any():
            nib.save(nib.Nifti1Image(m, img.affine, img.header), os.path.join(save_dir, f"rib_right_{i}.nii.gz"))

    print(f"[OK] {case} -> {save_dir}")
    return True

def process_all():
    case_ids = sorted({os.path.basename(f).split('_')[0] for f in glob(os.path.join(TEST_DIR, "*.nii.gz"))})
    ok = err = 0
    for c in case_ids:
        try:
            ok += 1 if process_case(c) else 0
        except Exception as e:
            err += 1
            print(f"[ERR] {c}: {e}")
    print(f"done ok={ok} err={err}")

if __name__ == "__main__":
    process_all()


[OK] CF5008 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5008
[OK] CF5009 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5009
[OK] CF5015 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5015
[OK] CF5017 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF5017
[OK] CF6008 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF6008
[OK] CF6011 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF6011
[OK] CF6028 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF6028
[OK] CF6031 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF6031
[OK] CF7011 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CF7011
[OK] CM5115 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CM5115
[OK] CM6116 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CM6116
[OK] CM7104 -> /home/cuixing/MDP/Data_Predict/Predicted_Result/50_Predict/CM7104
[OK] CM7109 -> /home/cuixing